Для выполнения домашнего задания был выбран dataset [Telco Customer Churn](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
df = pd.read_csv("dataset.csv", sep=",")
df.head(50)

In [ ]:
df.info()

Идентификатор клиента не влияет на отток, поэтому можем удалить его. 

In [ ]:
df.drop(columns=["customerID"], inplace=True)

In [ ]:
df["TotalCharges"].value_counts()

In [ ]:
df["TotalCharges_float"] = pd.to_numeric(df["TotalCharges"], errors="coerce").fillna(-999)


In [ ]:
df.select_dtypes(include='object')

In [ ]:
label_encoder = LabelEncoder()
columns = ["gender", "Partner"]
df[columns] = label_encoder.fit_transform(df[columns])
df